# Chapter 06-02: 단어 임베딩 (Word Embeddings)

## 학습 목표
- One-Hot 인코딩의 한계를 이해한다.
- 단어 임베딩의 개념과 장점을 파악한다.
- `tf.keras.layers.Embedding` 레이어를 구성하고 활용한다.
- 사전 학습된 임베딩(GloVe 등)을 불러오는 방법을 익힌다.
- 코사인 유사도를 통해 단어 간 의미적 유사도를 계산한다.

## 목차
1. [기본 임포트](#1.-기본-임포트)
2. [코사인 유사도 수식](#2.-코사인-유사도-수식)
3. [One-Hot 인코딩의 한계](#3.-One-Hot-인코딩의-한계)
4. [Embedding 레이어](#4.-Embedding-레이어)
5. [사전 학습 임베딩 (GloVe 시뮬레이션)](#5.-사전-학습-임베딩)
6. [코사인 유사도 계산](#6.-코사인-유사도-계산)
7. [정리](#7.-정리)

---
### 🐣 초등학생을 위한 단어 임베딩 친절 설명!

#### 🤔 컴퓨터는 단어를 어떻게 이해해요?

컴퓨터는 숫자만 알아요! 그래서 단어를 숫자로 바꿔야 해요.

#### 1️⃣ 기존 방법: One-Hot 인코딩

```
어휘: [고양이, 강아지, 자동차, 비행기]

고양이 = [1, 0, 0, 0]
강아지 = [0, 1, 0, 0]
자동차 = [0, 0, 1, 0]
```

❌ **문제**: 고양이와 강아지가 자동차만큼 '다르게' 표현돼요!
같은 동물인데도 컴퓨터는 구분을 못해요.

#### 2️⃣ 더 좋은 방법: 단어 임베딩(Word Embedding)

단어를 **의미가 담긴 좌표(벡터)**로 표현해요!

```
              동물성  크기  속도  ...  (50~300개 숫자!)
고양이 =       [0.9,  0.3, 0.4, ...]
강아지 =       [0.9,  0.4, 0.5, ...]  ← 고양이와 비슷!
자동차 =       [0.0,  0.7, 0.9, ...]  ← 완전 다름!
```

#### 📍 단어를 공간(지도)에 배치하는 것과 같아요!

```
높은 동물성 ↑
           고양이 • 강아지     ← 동물 영역
           새 •
           
           자동차 •  비행기 •  ← 탈것 영역
낮은 동물성 ↓
           낮은 속도 ←——→ 높은 속도
```

> 💡 **비유**: 세계지도에서
> 가까운 나라들은 비슷한 문화/언어를 가지듯,
> 임베딩 공간에서 가까운 단어들은 의미가 비슷해요!

#### 🔢 코사인 유사도로 '얼마나 비슷한지' 측정

$$\cos(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

- 1에 가까울수록 = 의미가 비슷한 단어
- 0에 가까울수록 = 관련 없는 단어
- -1에 가까울수록 = 반대 의미의 단어

#### ✨ 임베딩의 놀라운 특성

잘 학습된 임베딩은 이런 연산도 가능해요!

$$\vec{\text{왕}} - \vec{\text{남자}} + \vec{\text{여자}} \approx \vec{\text{여왕}}$$

> 💡 단어의 **의미**가 벡터 공간에 수학적으로 인코딩된 거예요!

In [ ]:
# 기본 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정 (macOS)
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

print(f"TensorFlow 버전: {tf.__version__}")
print(f"NumPy 버전: {np.__version__}")

## 2. 코사인 유사도 수식

두 벡터 $A$와 $B$ 사이의 **코사인 유사도**는 벡터 간의 각도로 유사성을 측정한다.

$$\cos(A, B) = \frac{A \cdot B}{\|A\| \|B\|}$$

- $A \cdot B$: 두 벡터의 내적 (dot product)
- $\|A\|$, $\|B\|$: 각 벡터의 L2 노름 (크기)

### 해석
| 코사인 유사도 값 | 의미 |
|-----------------|------|
| $1.0$ | 완전히 동일한 방향 (매우 유사) |
| $0.0$ | 직교 (관련 없음) |
| $-1.0$ | 완전히 반대 방향 (상반됨) |

임베딩 공간에서 의미적으로 유사한 단어들은 코사인 유사도가 높다.  
예: $\cos(\vec{\text{king}}, \vec{\text{queen}}) \approx 0.8$

## 3. One-Hot 인코딩의 한계

One-Hot 인코딩은 각 단어를 하나의 차원만 1이고 나머지는 0인 벡터로 표현한다.

### 문제점

1. **고차원성**: 어휘 사전 크기가 10만 개면 벡터 차원도 10만 차원
2. **희소성(Sparsity)**: 대부분의 값이 0 → 계산 비효율
3. **의미 부재**: 모든 단어 쌍의 거리가 동일 → 유사한 단어 관계 표현 불가

예를 들어, "cat"과 "dog"는 모두 동물이지만 One-Hot 표현에서는:
- cat = [1, 0, 0, 0, 0, ...]
- dog = [0, 1, 0, 0, 0, ...]
- car = [0, 0, 1, 0, 0, ...]

$\cos(\vec{\text{cat}}, \vec{\text{dog}}) = 0$, $\cos(\vec{\text{cat}}, \vec{\text{car}}) = 0$ → 구분 불가

In [ ]:
# One-Hot 인코딩의 한계 시각화

# 예시 어휘 사전
vocab = ["cat", "dog", "car", "truck", "kitten", "puppy"]
vocab_size = len(vocab)
word_to_idx = {w: i for i, w in enumerate(vocab)}

# One-Hot 벡터 생성
def one_hot(word, vocab_size, word_to_idx):
    """단어를 One-Hot 벡터로 변환"""
    vec = np.zeros(vocab_size)
    vec[word_to_idx[word]] = 1.0
    return vec

# 코사인 유사도 계산 함수
def cosine_similarity(a, b):
    """두 벡터 간의 코사인 유사도 계산"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)

# One-Hot 벡터 계산
cat_oh    = one_hot("cat",    vocab_size, word_to_idx)
dog_oh    = one_hot("dog",    vocab_size, word_to_idx)
kitten_oh = one_hot("kitten", vocab_size, word_to_idx)
car_oh    = one_hot("car",    vocab_size, word_to_idx)

print("=== One-Hot 벡터 ===")
print(f"  cat   : {cat_oh}")
print(f"  dog   : {dog_oh}")
print(f"  kitten: {kitten_oh}")
print(f"  car   : {car_oh}")
print()

print("=== One-Hot 코사인 유사도 ===")
print(f"  cat vs dog   : {cosine_similarity(cat_oh, dog_oh):.4f}  (동물끼리 → 0이어야 하지 않음!)")
print(f"  cat vs kitten: {cosine_similarity(cat_oh, kitten_oh):.4f}  (cat-kitten 매우 유사한데 0!)")
print(f"  cat vs car   : {cosine_similarity(cat_oh, car_oh):.4f}  (완전 다른데도 0)")
print()
print("→ One-Hot은 모든 단어 쌍의 코사인 유사도가 0 → 의미 구분 불가")

# One-Hot 행렬 시각화
fig, ax = plt.subplots(figsize=(8, 4))
oh_matrix = np.array([one_hot(w, vocab_size, word_to_idx) for w in vocab])
im = ax.imshow(oh_matrix, cmap='Blues', aspect='auto')
ax.set_xticks(range(vocab_size))
ax.set_xticklabels(vocab, rotation=45)
ax.set_yticks(range(vocab_size))
ax.set_yticklabels(vocab)
ax.set_title("One-Hot 인코딩 행렬\n(어휘 크기가 커지면 행렬도 거대해짐)")
plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

## 4. Embedding 레이어

`tf.keras.layers.Embedding`은 정수 인덱스를 밀집(dense) 벡터로 변환하는 레이어이다.  
내부적으로 **가중치 행렬** $W \in \mathbb{R}^{\text{vocab\_size} \times \text{embed\_dim}}$을 학습한다.

### 주요 파라미터

| 파라미터 | 설명 |
|----------|------|
| `input_dim` | 어휘 사전 크기 (정수 인덱스 최대값 + 1) |
| `output_dim` | 임베딩 벡터 차원 수 |
| `embeddings_initializer` | 가중치 초기화 방법 (기본: 균일 분포) |
| `mask_zero` | 패딩 마스킹 활성화 여부 |
| `trainable` | 가중치 업데이트 여부 (사전 학습 임베딩 고정 시 False) |

In [ ]:
# Embedding 레이어 실습

# 어휘 사전 크기와 임베딩 차원 설정
VOCAB_SIZE = 1000   # 어휘 사전 크기
EMBED_DIM  = 16     # 임베딩 벡터 차원

# Embedding 레이어 생성
embedding_layer = tf.keras.layers.Embedding(
    input_dim=VOCAB_SIZE,              # 어휘 사전 크기
    output_dim=EMBED_DIM,              # 임베딩 차원
    embeddings_initializer='uniform',  # 초기화: 균일 분포
    mask_zero=True,                    # 패딩 마스킹 활성화
    name='word_embedding'
)

# 입력 형태: (배치 크기, 시퀀스 길이)
# 출력 형태: (배치 크기, 시퀀스 길이, 임베딩 차원)
sample_input = tf.constant([[1, 2, 3, 4, 0],   # 배치 샘플 1 (0은 패딩)
                             [5, 6, 7, 0, 0]])  # 배치 샘플 2
output = embedding_layer(sample_input)

print("입력 형태:", sample_input.shape, "→ (배치 크기, 시퀀스 길이)")
print("출력 형태:", output.shape, "→ (배치 크기, 시퀀스 길이, 임베딩 차원)")
print()
print("임베딩 가중치 행렬 크기:", embedding_layer.embeddings.shape)
print(f"  → {VOCAB_SIZE}개 단어 × {EMBED_DIM}차원")
print()
print("단어 인덱스 1의 임베딩 벡터:", output[0][0].numpy())

## 5. 사전 학습 임베딩 (GloVe 시뮬레이션)

**GloVe(Global Vectors for Word Representation)**는 대규모 텍스트 코퍼스에서  
사전 학습된 단어 임베딩이다. 실무에서 자주 사용하는 방법은 다음과 같다:

1. GloVe 파일(`.txt`)을 다운로드한다.
2. 각 단어의 임베딩 벡터를 읽어 딕셔너리에 저장한다.
3. 자신의 어휘 사전에 맞는 임베딩 행렬을 구성한다.
4. Embedding 레이어 가중치를 해당 행렬로 초기화하고 `trainable=False`로 고정한다.

아래 코드는 GloVe 로드 과정을 **시뮬레이션**한다.

In [ ]:
# 사전 학습 임베딩 가중치 직접 설정 (GloVe 로드 시뮬레이션)

# 시뮬레이션용 어휘 사전과 임베딩 차원
words = ["cat", "dog", "kitten", "puppy", "car", "truck"]
SIM_VOCAB_SIZE = len(words) + 2  # 패딩(0) + OOV(1) + 실제 단어들
SIM_EMBED_DIM  = 8

word_to_idx_sim = {word: idx + 2 for idx, word in enumerate(words)}  # 0=패딩, 1=OOV

# GloVe 임베딩 시뮬레이션
# 실제에서는 파일에서 읽어오지만, 여기서는 의미 있는 값을 수동으로 정의
# 차원 의미 (임의 설정): [동물성, 작음, 털, 반려, 기계, 무거움, 속도, 육지]
simulated_glove = {
    "cat":    np.array([0.9,  0.6,  0.9,  0.8, -0.5, -0.3,  0.2,  0.4]),
    "dog":    np.array([0.9,  0.5,  0.8,  0.9, -0.4, -0.2,  0.4,  0.5]),
    "kitten": np.array([0.8,  0.9,  0.9,  0.7, -0.5, -0.4,  0.1,  0.3]),
    "puppy":  np.array([0.8,  0.8,  0.7,  0.9, -0.4, -0.3,  0.2,  0.4]),
    "car":    np.array([-0.2, 0.3, -0.3, -0.1,  0.9,  0.5,  0.8,  0.6]),
    "truck":  np.array([-0.1, 0.1, -0.2, -0.2,  0.9,  0.9,  0.6,  0.8]),
}

# 임베딩 행렬 초기화 (패딩과 OOV 토큰은 0 벡터)
embedding_matrix = np.zeros((SIM_VOCAB_SIZE, SIM_EMBED_DIM))
for word, idx in word_to_idx_sim.items():
    if word in simulated_glove:
        embedding_matrix[idx] = simulated_glove[word]

print("임베딩 행렬 크기:", embedding_matrix.shape)
print()

# Embedding 레이어에 사전 학습 가중치 적용
pretrained_embedding = tf.keras.layers.Embedding(
    input_dim=SIM_VOCAB_SIZE,
    output_dim=SIM_EMBED_DIM,
    # 사전 학습 가중치를 초기값으로 사용
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,  # 사전 학습 가중치 고정 (미세 조정 시 True로 변경)
    name='pretrained_glove'
)

# 단어 인덱스로 임베딩 벡터 조회
cat_idx = word_to_idx_sim["cat"]
cat_embed = pretrained_embedding(tf.constant([[cat_idx]]))
print(f"'cat' (인덱스 {cat_idx}) 임베딩 벡터:")
print(f"  {cat_embed.numpy()[0][0]}")

## 6. 코사인 유사도 계산

In [ ]:
# 임베딩 벡터 코사인 유사도 계산 예시

def cosine_similarity(a, b):
    """두 벡터의 코사인 유사도 계산"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)

# 각 단어의 임베딩 벡터 추출
word_vectors = {word: simulated_glove[word] for word in words}

# 유사도 행렬 계산
n = len(words)
sim_matrix = np.zeros((n, n))
for i, w1 in enumerate(words):
    for j, w2 in enumerate(words):
        sim_matrix[i, j] = cosine_similarity(word_vectors[w1], word_vectors[w2])

# 유사도 행렬 시각화
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(sim_matrix, cmap='RdYlGn', vmin=-1, vmax=1)
ax.set_xticks(range(n))
ax.set_xticklabels(words, rotation=45, ha='right')
ax.set_yticks(range(n))
ax.set_yticklabels(words)
ax.set_title("단어 임베딩 코사인 유사도 행렬\n(초록=높은 유사도, 빨강=낮은 유사도)")
plt.colorbar(im, ax=ax)

# 유사도 값 표시
for i in range(n):
    for j in range(n):
        ax.text(j, i, f"{sim_matrix[i,j]:.2f}",
                ha='center', va='center', fontsize=9,
                color='black')

plt.tight_layout()
plt.show()

print("\n주목할 유사도:")
pairs = [
    ("cat",    "kitten", "비슷한 동물"),
    ("dog",    "puppy",  "비슷한 동물"),
    ("car",    "truck",  "비슷한 탈것"),
    ("cat",    "car",    "완전히 다른 범주"),
]
for w1, w2, desc in pairs:
    sim = cosine_similarity(word_vectors[w1], word_vectors[w2])
    print(f"  {w1:8s} vs {w2:8s} ({desc:15s}): {sim:.4f}")

## 7. 정리

### 핵심 개념 요약

| 방법 | 차원 | 의미 표현 | 특징 |
|------|------|-----------|------|
| **One-Hot** | 어휘 크기 | 불가 | 희소, 고차원, 계산 비효율 |
| **Word Embedding** | 수십~수백 | 가능 | 밀집, 저차원, 의미적 유사도 포착 |
| **사전 학습 임베딩** | 수백 | 매우 우수 | 대규모 학습, 전이 학습 |

### 임베딩 공간의 특성
잘 학습된 임베딩은 다음과 같은 벡터 연산이 가능하다:

$$\vec{\text{king}} - \vec{\text{man}} + \vec{\text{woman}} \approx \vec{\text{queen}}$$

### 다음 챕터 예고
- **Chapter 06-03**: RNN, LSTM, GRU  
  시퀀스 데이터를 처리하는 순환 신경망 구조를 학습한다.